In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
data = pd.read_excel('女网球q1.xlsx')

# 定义p1和p2的列
p1_columns = [
    'p1得分', 'p1累计得分次数', 'p1_ace（p1发球制胜球）', 'p1_winner(p1击球制胜球)', 'p1发球失误率', 
    'p1_unf_err（非受迫性失误）', 'p1_break（赛点得分）', 'p1_distance_run（跑动距离）', 'p1_speed_mph（p1发球速度）'
]
p2_columns = [
    'p2得分', 'p2累计得分次数', 'p2_ace（p2发球制胜球）', 'p2_winner(p2击球制胜球)', 'p2发球失误率', 
    'p2_unf_err（非受迫性失误）', 'p2_break（赛点得分）', 'p2_distance_run（跑动距离）', 'p2_speed_mph（p2发球速度）'
]

# 权重
weights = np.array([0.25, 0.05, 0.15, 0.15, -0.15, -0.15, 0.1, 0.05, 0.05])

# 标准化处理
scaler = MinMaxScaler()
data_p1_normalized = scaler.fit_transform(data[p1_columns])
data_p2_normalized = scaler.fit_transform(data[p2_columns])

# 负向指标逆向处理
for i in [4, 5]:  # 发球失误率和非受迫性失误的索引
    data_p1_normalized[:, i] = 1 - data_p1_normalized[:, i]
    data_p2_normalized[:, i] = 1 - data_p2_normalized[:, i]

# 计算加权标准化矩阵
p1_weighted = data_p1_normalized * weights
p2_weighted = data_p2_normalized * weights

# 确定正理想解和负理想解
positive_ideal_p1 = np.max(p1_weighted, axis=0)
negative_ideal_p1 = np.min(p1_weighted, axis=0)
positive_ideal_p2 = np.max(p2_weighted, axis=0)
negative_ideal_p2 = np.min(p2_weighted, axis=0)

# 计算到理想解的距离
distance_positive_p1 = np.sqrt(np.sum((p1_weighted - positive_ideal_p1) ** 2, axis=1))
distance_negative_p1 = np.sqrt(np.sum((p1_weighted - negative_ideal_p1) ** 2, axis=1))
distance_positive_p2 = np.sqrt(np.sum((p2_weighted - positive_ideal_p2) ** 2, axis=1))
distance_negative_p2 = np.sqrt(np.sum((p2_weighted - negative_ideal_p2) ** 2, axis=1))

# 计算相对接近度
closeness_p1 = distance_negative_p1 / (distance_positive_p1 + distance_negative_p1)
closeness_p2 = distance_negative_p2 / (distance_positive_p2 + distance_negative_p2)

# 将评分添加到原始数据集中
data['p1_comprehensive_score'] = closeness_p1
data['p2_comprehensive_score'] = closeness_p2



# 保存数据集到新的Excel文件
data.to_excel('评分.xlsx', index=False)  
